Info about LIME and more examples can be found here: https://github.com/marcotcr/lime/tree/master

In [ ]:
%pip install lime

In [ ]:
from lime import lime_image
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
from skimage.color import rgb2gray

In [ ]:
model = tf.keras.models.load_model('/path/to/model')

In [ ]:
X = np.load('/path/to/xData.npy')
y = np.load('/path/to/yData.npy')

X = X/255.0

In [ ]:
def make_prediction(img):
    # LIME treats the input as 2D RGB images, so we  convert the img to grayscale here so that the image is compatible with the model
    gray_img = rgb2gray(img).reshape(-1, 128, 128, 1) # Modify this to match the input shape for your model. The input for the one used to create this notebook was (128,128,1)
    return model.predict(gray_img)

In [ ]:
explainer = lime_image.LimeImageExplainer()

In [ ]:
explanation = explainer.explain_instance(X[0].squeeze(), make_prediction, num_samples=1000)

In [ ]:
# Get the explanation for a specific class/prediction (indicated by explanation.top_labels[X])
# Green areas mark areas that contribute positively to predicting the specified class (important for predicting that class)
# Red areas mark areas that contribute negatively to predicting the specified class (important for predicting that the image belongs to a different specific class, or important for predicting that the image does NOT belong to the specified class)

temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [ ]:
# Plot explanation weights onto a heatmap visualization so that importance of different regions can be compared to each other

dict_heatmap = dict(explanation.local_exp[explanation.top_labels[0]])
heatmap = np.vectorize(dict_heatmap.get)(explanation.segments) 

plt.imshow(heatmap, cmap = 'RdBu', vmin  = -heatmap.max(), vmax = heatmap.max())
plt.colorbar()